<a href="https://colab.research.google.com/github/ungvietanh20172394/COF-Tank/blob/main/cof_shell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# _C1 = 31623.0
# _C10 = 9.744
# _C11 = 0.145
_C12 = 1.8
C13 = 6.29
# _C14 = 1.0
# _C15 = 4.685
# _C16 = 30.89
# _C17 = 0.001481
# _C18 = 0.005
# _C19 = 1.085
# _C2 = 1000.0
# _C20 = 1.013
# _C21 = 5328.0
# _C22 = 5.8
# _C23 = 0.45
# _C24 = 2.6
# _C25 = 0.0296 
# _C26 = 100.0 
# _C27 = 1.0 
# _C28 = 1000.0 
# _C29 = 0.0004303 
# _C3 = 4536.0 
# _C30 = 9.76E-08 
# _C31 = 864.0
# _C32 = 0.543
# _C33 = 0.0815
# _C34 = 86.4 
# _C35 = 2.382
# _C36 = 30.5
# _C37 = 1.408E-08
# _C38 = 1.1341
# _C39 = 3.9365
# _C4 = 2.205 
# _C40 = 5.9352 
# _C5 = 25.2 
# _C6 = 55.6 
# _C7 = 1.0 
# _C8 = 0.0929
# _C9 = 0.123

In [ ]:
class CA_SHELL: #datsua
    def __init__(self, FLUID, FLUID_HEIGHT, SHELL_COURSE_HEIGHT, TANK_DIAMETER, EnvironSensitivity,
                 P_lvdike, P_onsite, P_offsite, MATERIAL_COST, API_COMPONENT_TYPE_NAME, PRODUCTION_COST,Soil_type,
                 TANK_FLUID,CHT,PROD_COST,EQUIP_OUTAGE_MULTIPLIER,EQUIP_COST,POP_DENS,INJ_COST,release_Fluid_Percent_Toxic,FLUID_PHASE,proposalID):
        self.FLUID = FLUID
        self.FLUID_HEIGHT = FLUID_HEIGHT
        self.SHELL_COURSE_HEIGHT = SHELL_COURSE_HEIGHT
        self.TANK_DIAMETER = TANK_DIAMETER
        self.EnvironSensitivity = EnvironSensitivity
        self.P_lvdike = P_lvdike
        self.P_onsite = P_onsite
        self.P_offsite = P_offsite
        self.MATERIAL_COST = MATERIAL_COST
        self.API_COMPONENT_TYPE_NAME = API_COMPONENT_TYPE_NAME
        self.PRODUCTION_COST = PRODUCTION_COST
        self.Soil_type = Soil_type
        self.TANK_FLUID=TANK_FLUID
        self.CHT=CHT
        # self.EQUIPMENT_COST = EQUIPMENT_COST
        #bổ sung 5 tham số đầu vào cho TankShell
        self.PROD_COST = PROD_COST
        self.EQUIP_OUTAGE_MULTIPLIER = EQUIP_OUTAGE_MULTIPLIER
        self.EQUIP_COST = EQUIP_COST
        self.POP_DENS = POP_DENS
        self.INJ_COST = INJ_COST
        self.release_Fluid_Percent_Toxic = release_Fluid_Percent_Toxic/1000000
        self.FLUID_PHASE = FLUID_PHASE
        self.proposalID = proposalID

    def NBP(self):
        data = DAL_CAL.POSTGRESQL.GET_TBL_52(self.FLUID)
        nbp  = ((data[2] -32)/1.8)
        return nbp

    def ambient(self):
        return DAL_CAL.POSTGRESQL.GET_RELEASE_PHASE(self.FLUID)

    def ReleasePhase(self):
        try:
            if (self.FLUID_PHASE=="Gas"):
                return "Gas"
            elif (self.ambient()=="Liquid") and (self.FLUID_PHASE=="Liquid"):
                return "Liquid"
            elif self.NBP()<=300:
                return "Liquid"
            else:
                return "Gas"
        except Exception as e:
            print(e)
            return 0


    def FC_Category(self, fc):
        if (fc <= 10000):
            return "A"
        elif (fc <= 100000):
            return "B"
        elif (fc <= 1000000):
            return "C"
        elif (fc <= 10000000):
            return "D"
        else:
            return "E"

    def k_h_bottom(self):
        k_h = [0, 0, 0]
        if (self.Soil_type == "Coarse Sand"):
            k_h[0] = 0.1
            k_h[1] = 0.01
            k_h[2] = 0.33
        elif(self.Soil_type == "Fine Sand"):
            k_h[0] = 0.01
            k_h[1] = 0.001
            k_h[2] = 0.33
        elif(self.Soil_type == "Very Fine Sand"):
            k_h[0] = pow(10, -3)
            k_h[1] = pow(10, -5)
            k_h[2] = 0.33
        elif(self.Soil_type == "Silt"):
            k_h[0] = pow(10, -5)
            k_h[1] = pow(10, -6)
            k_h[2] = 0.41
        elif(self.Soil_type == "Sandy Clay"):
            k_h[0] = pow(10, -6)
            k_h[1] = pow(10, -7)
            k_h[2] = 0.45
        elif(self.Soil_type == "Clay"):
            k_h[0] = pow(10, -7)
            k_h[1] = pow(10, -8)
            k_h[2] = 0.5
        elif(self.Soil_type == "Concrete-Asphalt"):
            k_h[0] = pow(10, -10)
            k_h[1] = pow(10, -11)
            k_h[2] = 0.3
        else:
            k_h[0] = 1
            k_h[1] = 0.1
            k_h[2] = 0.4
        return k_h

    def k_h_water(self):
        k_h = self.k_h_bottom()
        return C31 * (k_h[0] + k_h[1]) / 2

    def GET_PL_UL_SHELL(self,select):
        data = [0,0]
        if (self.TANK_FLUID == "Gasoline"):
            data[0] = 684.018
            data[1] = 4.01 * pow(10, -3)
        elif(self.TANK_FLUID == "Light Diesel Oil"):
            data[0] = 734.011
            data[1] = 1.04 * pow(10, -3)
        elif(self.TANK_FLUID == "Heavy Diesel Oil"):
            data[0] = 764.527
            data[1] = 2.46 * pow(10, -3)
        elif(self.TANK_FLUID == "Fuel Oil"):
            data[0] = 775.019
            data[1] = 3.69 * pow(10, -2)
        elif(self.TANK_FLUID == "Crude Oil"):
            data[0] = 775.019
            data[1] = 3.69 * pow(10, -2)
        elif (self.TANK_FLUID == "Heavy Crude Oil"):
            data[0] = 900.026
            data[1] = 4.6 * pow(10, -2)
        elif (self.TANK_FLUID == "Heavy Fuel Oil"):
            data[0] = 900.026
            data[1] = 4.6 * pow(10, -2)
        else:
            data[0] = 1000
            data[1] = 1
        return data[select]

    def k_h_prod(self):
        return self.k_h_water() * (self.GET_PL_UL_SHELL(0) / 1000) * (1 / self.GET_PL_UL_SHELL(1))

    def vel_s_prod(self):
        kh = self.k_h_bottom()
        return self.k_h_prod() / kh[2]

    def d_n_shell(self, i):
        if (i == 1):
            dn = 3.175
        elif (i == 2):
            dn = 6.35
        elif (i == 3):
            dn = 50.8
        else:
            dn = 250*self.TANK_DIAMETER
        return dn

    def a_n_shell(self, i):
        return math.pi * pow(self.d_n_shell(i),2) / 4

    def LHT_above(self):
        try:
            LHTab = self.FLUID_HEIGHT-(self.get_Course()-1)*self.CHT
            if LHTab>=0:
                return LHTab
            else:
                return 0
        except Exception as e:
            print(e)
            return 0

    def get_Course(self):
        if self.API_COMPONENT_TYPE_NAME=="COURSE-1":
            return 1
        if self.API_COMPONENT_TYPE_NAME=="COURSE-2":
            return 2
        if self.API_COMPONENT_TYPE_NAME=="COURSE-3":
            return 3
        if self.API_COMPONENT_TYPE_NAME=="COURSE-4":
            return 4
        if self.API_COMPONENT_TYPE_NAME=="COURSE-5":
            return 5
        if self.API_COMPONENT_TYPE_NAME=="COURSE-6":
            return 6
        if self.API_COMPONENT_TYPE_NAME=="COURSE-7":
            return 7
        if self.API_COMPONENT_TYPE_NAME=="COURSE-8":
            return 8
        if self.API_COMPONENT_TYPE_NAME=="COURSE-9":
            return 9
        if self.API_COMPONENT_TYPE_NAME=="COURSE-10":
            return 10

    def Lvol_abouve(self):
        return math.pi * pow(self.TANK_DIAMETER,2) * self.LHT_above() / 4

    def W_n_Tank(self, i):
        try:
            if(self.MATERIAL_COST==1):
                return C32 * 0.61 * self.a_n_shell(i) * math.sqrt(2 * 9.8196 * self.FLUID_HEIGHT)
            else:
                return C32 * 0.61 * self.a_n_shell(i) * math.sqrt(2 * 9.8196 * self.FLUID_HEIGHT)
        except Exception as e:
            print(e)
            return 0

    def Bbl_total_shell(self):
        try:
            return math.pi * pow(self.TANK_DIAMETER,2) * self.FLUID_HEIGHT / (4* C13)
        except Exception as e:
            print(e)
            return 0

    def Bbl_avail(self, i):
        try:
            return self.Lvol_abouve()*C13
            # return math.pi * pow(self.TANK_DIAMETER, 2) * (self.FLUID_HEIGHT - (i - 1) * self.SHELL_COURSE_HEIGHT) / (4 * C13)
        except Exception as e:
            print(e)
            return 0

    def ld_tank(self, i):
        try:
            if(self.d_n_shell(i) <= 3.175):
                return min(self.Bbl_avail(i) / self.W_n_Tank(i), 7)
            else:
                # return 1
                return min(self.Bbl_avail(i) / self.W_n_Tank(i), 1)
        except:
            return 1

    def Bbl_leak_n(self, i):
        return min(self.W_n_Tank(i) * self.ld_tank(i), self.Bbl_avail(i))

    def getCost(self):
        try:
            costTANK = [0, 0, 0, 0, 0, 0]
            if (self.EnvironSensitivity == "High"):
                costTANK[0] = 10
                costTANK[1] = 50
                costTANK[2] = 500
                costTANK[3] = 3000
                costTANK[4] = 10000
                costTANK[5] = 5000
            elif(self.EnvironSensitivity == "Medium"):
                costTANK[0] = 10
                costTANK[1] = 50
                costTANK[2] = 250
                costTANK[3] = 1500
                costTANK[4] = 5000
                costTANK[5] = 1500
            elif(self.EnvironSensitivity == "Low"):
                costTANK[0] = 10
                costTANK[1] = 50
                costTANK[2] = 100
                costTANK[3] = 500
                costTANK[4] = 1000
                costTANK[5] = 500
            else:
                costTANK[0] = 0
                costTANK[1] = 0
                costTANK[2] = 0
                costTANK[3] = 0
                costTANK[4] = 0
                costTANK[5] = 0
            return costTANK
        except Exception as e:
            print(e)
            return 0

    def Bbl_leak_release(self):
        try:
            obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
            summ = self.Bbl_leak_n(1) * obj[0] + self.Bbl_leak_n(2) * obj[1] + self.Bbl_leak_n(3) * obj[2]
            return summ / obj[4]
        except Exception as e:
            print(e)
            return 0

    def Bbl_leak_indike(self):
        try:
            return self.Bbl_leak_release() * (1 - self.P_lvdike / 100)
        except Exception as e:
            print(e)
            return 0

    def Bbl_leak_ssonsite(self):
        try:
            return self.P_onsite * (self.Bbl_leak_release() - self.Bbl_leak_indike()) / 100
        except Exception as e:
            print(e)
            return 0

    def Bbl_leak_ssoffsite(self):
        try:
            return self.P_offsite * (self.Bbl_leak_release() - self.Bbl_leak_indike() - self.Bbl_leak_ssonsite()) / 100
        except Exception as e:
            print(e)
            return 0

    def Bbl_leak_water(self):
        try:
            return self.Bbl_leak_release() - (self.Bbl_leak_indike() + self.Bbl_leak_ssonsite() + self.Bbl_leak_ssoffsite())
        except Exception as e:
            print(e)
            return 0

    def FC_leak_environ(self):
        try:
            cost = self.getCost()
            return self.Bbl_leak_indike() * cost[0] + self.Bbl_leak_ssonsite() * cost[1] + self.Bbl_leak_ssoffsite() * cost[2] + self.Bbl_leak_water() * cost[5]
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_n(self):
        try:
            return self.Bbl_avail(4)
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_release(self):
        try:
            obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
            return self.Bbl_rupture_n() * obj[3] / obj[4]
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_indike(self):
        try:
            return self.Bbl_rupture_release() * (1 - self.P_lvdike / 100)
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_ssonsite(self):
        try:
            return self.P_onsite * (self.Bbl_rupture_release() - self.Bbl_rupture_indike()) / 100
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_ssoffsite(self):
        try:
            return self.P_offsite * (self.Bbl_rupture_release() - self.Bbl_rupture_indike() - self.Bbl_rupture_ssonsite()) / 100
        except Exception as e:
            print(e)
            return 0

    def Bbl_rupture_water(self):
        try:
            return self.Bbl_rupture_release() - (self.Bbl_rupture_indike() + self.Bbl_rupture_ssonsite() + self.Bbl_rupture_ssoffsite())
        except Exception as e:
            print(e)
            return 0

    def FC_rupture_environ(self):
        try:
            cost = self.getCost()
            return self.Bbl_rupture_indike() * cost[0] + self.Bbl_rupture_ssonsite() * cost[1] + self.Bbl_rupture_ssoffsite() * cost[2] + self.Bbl_rupture_water() * cost[5]
        except Exception as e:
            print(e)
            return 0

    def FC_environ_shell(self):
        try:
            return self.FC_leak_environ() + self.FC_rupture_environ()
        except Exception as e:
            print(e)
            return 0

    def FC_PROD_SHELL(self):
        try:
            obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
            t = obj[0] * obj[9] + obj[1] * obj[10] + obj[2] * obj[11] + obj[3] * obj[12]
            return t * self.PRODUCTION_COST / obj[4]
        except:
            return 0

    def fc_cmd(self):
        try:
            obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
            t = obj[0] * obj[5] + obj[1] * obj[6] + obj[2] * obj[7] + obj[3] * obj[8]
            fc_cmd = t * self.MATERIAL_COST / obj[4]
            return fc_cmd
        except Exception as e:
            print(e)
            return 0
    #bổ sung tính 3 tham số đầu ra cho tankshell
    def fc_affa_tank(self): #dat
        if self.release_Fluid_Percent_Toxic<=0:
            return 0
        else:
            cacmd =self.CA_Flam_Cmd_shell()
            fc_affa = cacmd * self.EQUIP_COST
            return fc_affa
    def  outage_affa_tank(self):
        fcaffa = abs(self.fc_affa_tank())
        if fcaffa != 0:
            b = 1.242 + 0.585 * math.log10(fcaffa * pow(10.0, -6.0))
        else:
            b = 0
        return pow(10.0, b)
    def ca_cmd_flame_shell(self):
        t=7*pow(10,-5)*self.AINL_Cmd(1)+2.5*pow(10,-5)*self.AINL_Cmd(2)+5*pow(10,-6)*self.AINL_Cmd(3)+ pow(10,-7)*self.AINL_Cmd(4)
        ca_cmd_flame = t/0.0001001
        return abs(ca_cmd_flame)

    def AINL_Cmd(self,i):
        AINL_Cmd =self.getEquationConstants(0)*pow(self.rate_Flammable(i),self.getEquationConstants(1))
        if (AINL_Cmd >0):
            return AINL_Cmd
        else:
            return 0

    def rate_Flammable(self,i):
        # rate_Flammable = (self.W_n_Tank(i)*1.84*pow(10,-6))*pl_ul[0]
        rate_Flammable = (self.W_n_Tank(i) * 1.84 * pow(10, -6)) * self.GET_PL_UL_SHELL(0)
        if(rate_Flammable>0):
            return rate_Flammable
        else:
            return 0

    def getEquationConstants(self, select):
        try:
            data1 = DAL_CAL.POSTGRESQL.GET_TBL_58(self.FLUID)
            data2 = DAL_CAL.POSTGRESQL.GET_TBL_59(self.FLUID)
            input = [0, 0, 0, 0, 0, 0, 0, 0]
            if (self.TANK_FLUID == "Water"):
                return 0
            else:
                input[0] = data1[2]
                input[1] = data1[3]
                input[2] = data1[6]
                input[3] = data1[7]
                input[4] = data2[2]
                input[5] = data2[3]
                input[6] = data2[6]
                input[7] = data2[7]
            return input[select]
        except Exception as e:
            return 0
            print(e)
            print('exception at getEquationConstants')

    def fc_prod_tank(self):
        return (CA_NORMAL.outage_cmd(self)*self.EQUIP_OUTAGE_MULTIPLIER +self.outage_affa_tank())*self.PROD_COST
    def fc_inj_tank(self):
        if self.release_Fluid_Percent_Toxic<=0:
            return 0
        else:
            return self.max_total_inj()*self.POP_DENS*self.INJ_COST
    def leakDurationToxic(self,i):
        if self.release_Fluid_Percent_Toxic==0:
            return 0
        else:
            volume_Fluid_Shell= self.Lvol_abouve()
            mass_Available=volume_Fluid_Shell*self.GET_PL_UL_SHELL(0)
            return min(mass_Available/self.rate_Flammable(i),3600.0)
    def ldToxicMinutes(self,i):
        if self.release_Fluid_Percent_Toxic==0:
            return 0
        else:
            a=max(self.leakDurationToxic(i)/60.0,5.0)
            b=min(a,60.0)
            return b
    def a_cmd(self, select): #done
        try:
            data = DAL_CAL.POSTGRESQL.GET_TBL_58(self.FLUID)
            a_cmd = [0,0,0,0]
            if(self.ReleasePhase() == "Gas"):
                a_cmd[0] = data[0]
                a_cmd[1] = data[4]
                a_cmd[2] = data[8]
                a_cmd[3] = data[12]
            else:
                a_cmd[0] = data[2]
                a_cmd[1] = data[6]
                a_cmd[2] = data[10]
                a_cmd[3] = data[14]
            if(a_cmd[select - 1] == 1):
                return 0
            else:
                return a_cmd[select - 1]
        except Exception as e:
            return 0
            print(e)
            print('exception at a_cmd')

    def b_cmd(self, select):
        try:
            data = DAL_CAL.POSTGRESQL.GET_TBL_58(self.FLUID)
            b_cmd = [0, 0, 0, 0]
            if (self.ReleasePhase() == "Gas"):
                b_cmd[0] = data[1]
                b_cmd[1] = data[5]
                b_cmd[2] = data[9]
                b_cmd[3] = data[13]
            else:
                b_cmd[0] = data[3]
                b_cmd[1] = data[7]
                b_cmd[2] = data[11]
                b_cmd[3] = data[15]
            return b_cmd[select - 1]
        except Exception as e:
            return 0
            print(e)
            print('exception at b_cmd')
    def a_inj(self, select): #done
        try:
            data = DAL_CAL.POSTGRESQL.GET_TBL_59(self.FLUID)
            a_inj = [0, 0, 0, 0]
            if(self.ReleasePhase() == "Gas"):
                a_inj[0] = data[0]
                a_inj[1] = data[4]
                a_inj[2] = data[8]
                a_inj[3] = data[12]
            else:
                a_inj[0] = data[2]
                a_inj[1] = data[6]
                a_inj[2] = data[10]
                a_inj[3] = data[14]
            return a_inj[select - 1]
        except Exception as e:
            return 0
            print(e)
            print('exception at a_inj')

    def b_inj(self, select): #done
        try:
            data = DAL_CAL.POSTGRESQL.GET_TBL_59(self.FLUID)
            b_inj = [0, 0, 0, 0]
            if (self.ReleasePhase() == "Gas"):
                b_inj[0] = data[1]
                b_inj[1] = data[5]
                b_inj[2] = data[9]
                b_inj[3] = data[13]
            else:
                b_inj[0] = data[3]
                b_inj[1] = data[7]
                b_inj[2] = data[11]
                b_inj[3] = data[15]
            return b_inj[select - 1]
        except Exception as e:
            return 0
            print(e)
            print('exception at b_inj')
    def fact_mit(self): #checked
        try:
            if (self.MITIGATION_SYSTEM == "Inventory blowdown, coupled with isolation system activated remotely or automatically"):
                return 0.25
            elif (self.MITIGATION_SYSTEM == "Fire water deluge system and monitors"):
                return 0.2
            elif (self.MITIGATION_SYSTEM == "Fire water monitors only"):
                return 0.05
            else:
                return 0.15
        except Exception as e:
            return 0
            print(e)
            print('exception at fact_mit')

    def TYPE_FLUID(self): #done
        try:
            API_TYPE = "TYPE 0"
            if (self.FLUID == ""):
                API_TYPE = "TYPE 0"
            else:
                if(self.FLUID == "C1-C2" or self.FLUID == "C13-C16" or self.FLUID == "C17-C25" or self.FLUID =="C25+" or
                   self.FLUID == "C3-C4" or self.FLUID == "C5" or self.FLUID == "C6-C8" or self.FLUID == "C9-C12" or
                   self.FLUID == "Acid" or self.FLUID == "AlCl3" or self.FLUID == "H2" or self.FLUID == "H2S" or self.FLUID == "HCl" or
                   self.FLUID == "HF" or self.FLUID == "Nitric Acid" or self.FLUID == "NO2" or self.FLUID == "Phosgene" or
                   self.FLUID == "Pyrophoric" or self.FLUID == "Steam" or self.FLUID == "TDI" or self.FLUID == "Water" or self.FLUID =="Ammonia"
                   or self.FLUID == "Chorine"):
                    API_TYPE = "TYPE 0"
                elif(self.FLUID == "CO" or self.FLUID == "DEE" or self.FLUID == "EE" or self.FLUID == "EEA" or self.FLUID == "EG" or
                     self.FLUID == "EO" or self.FLUID == "Methanol" or self.FLUID == "PO" or self.FLUID == "Styrene" or self.FLUID == "Aromatics"):
                    API_TYPE = "TYPE 1"
                else:
                    API_TYPE = "TYPE 0"
            return API_TYPE
        except Exception as e:
            return 0
            print(e)
            print('exception at type fluid')


    def ca_cmdn_ainl_cont(self, select , i): #done
        try:
            x = self.rate_Flammable(i)
            return (self.a_cmd(select) * pow(x, self.b_cmd(select)))
        except Exception as e:
            return 0
            print(e)
            print('exception at ca_cmdn_ainl_cont')


    def ca_cmdn_ail_cont(self, select , i): #done
        try:
            x = self.rate_Flammable(i)
            return (self.a_cmd(select) * pow(x, self.b_cmd(select)))
        except Exception as e:
            return 0
            print(e)
            print('exception at ca_cmdn_ail_cont')

    def ca_inj_ainl_cont(self, select , i): #done
        try:
            x = self.rate_Flammable(i)
            return (self.a_inj(select) * pow(x, self.b_inj(select)))
        except Exception as e:
            return 0
            print(e)
            print('exception at ca_inj_ainl_cont')


    def ca_inj_ail_cont(self, select , i): #done
        try:
            x = self.rate_Flammable(i)
            return (self.a_inj(select) * pow(x, self.b_inj(select)))
        except Exception as e:
            return 0
            print(e)
            print('exception at ca_inj_ail_cont')


    def fact_ait(self): #checked
        try:
            data = DAL_CAL.POSTGRESQL.GET_TBL_52(self.FLUID)
            ait = data[9]
            if ((self.STORED_TEMP + 55.6) <= ait):
                return 0
            elif((self.STORED_TEMP - 55.6) >= ait):
                return 1
            else:
                return (self.STORED_TEMP - ait + 55.6) / (2 * 55.6)
        except Exception as e:
            return 0
            print(e)
            print('exception at fact_ait')

    def CA_Flam_Cmd_shell(self):
        obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
        return (obj[0]*self.ca_cmdn_ail_cont(2,1)+obj[1]*self.ca_cmdn_ail_cont(2,2)+obj[2]*self.ca_cmdn_ail_cont(2,3)+obj[3]*self.ca_cmdn_ail_cont(2,4))/obj[4]

    def CA_Flam_inj_shell(self):
        try:
            obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
            return (obj[0]*self.ca_inj_ail_cont(2,1)+obj[1]*self.ca_inj_ail_cont(2,2)+obj[2]*self.ca_inj_ail_cont(2,3)+obj[3]*self.ca_inj_ail_cont(2,4))/obj[4]
        except Exception as e:
            print(e)
            return 0

    def ConstC(self,i):
        input = self.ldToxicMinutes(i)
        if (input >= 60):
            return 1.2266
        elif(input >= 40):
            return 1.2297
        elif(input >= 20):
            return 1.2370
        elif(input >= 10):
            return 1.2410
        elif(input >= 5):
            return 1.2411
        elif(input == 0):
            return 0
        else:
            return 0.9674
    def ConstD(self,i):
        input = self.ldToxicMinutes(i)
        if (input >= 60):
            return 4.4365
        elif(input >= 40):
            return 4.3626
        elif(input >= 20):
            return 4.238
        elif(input >= 10):
            return 4.0948
        elif(input >= 5):
            return 3.9686
        elif (input == 0):
            return 0
        else:
            return 2.7840

    def releaseRateMass(self,i):
        if (self.release_Fluid_Percent_Toxic*self.rate_Flammable(i))<=0:
            return 1
        else:
            return self.release_Fluid_Percent_Toxic*self.rate_Flammable(i)
    def toxic_Inj(self,i):
        if self.release_Fluid_Percent_Toxic==0:
            return 0
        else:
            t=0.0929 * pow(10.0,(self.ConstC(i)*math.log10(2.205*self.releaseRateMass(i))+self.ConstD(i)))
            return t
    def total_toxic_Inj(self):
        obj = DAL_CAL.POSTGRESQL.GET_API_COM(self.API_COMPONENT_TYPE_NAME)
        t = obj[0] * self.toxic_Inj(1) + obj[1] * self.toxic_Inj(2) + obj[2] * self.toxic_Inj(3) + obj[3] * self.toxic_Inj(4)
        total_toxic_Inj = t / obj[4]
        return abs(total_toxic_Inj)
    def max_total_inj(self):
        total_toxic_Inj=self.total_toxic_Inj()
        CA_Flam_inj_shell=self.CA_Flam_inj_shell()
        return max(total_toxic_Inj,CA_Flam_inj_shell)
    def ca_cmd_shell(self):
        CA_Flam_Cmd_shell = self.CA_Flam_Cmd_shell()
        CA_Flam_inj_shell = self.CA_Flam_inj_shell()
        return max(CA_Flam_Cmd_shell,CA_Flam_inj_shell)
    def FC_total_shell(self):
        ca_cmd_shell=self.ca_cmd_shell()
        total_toxic_Inj=self.total_toxic_Inj()
        return max(ca_cmd_shell,total_toxic_Inj)
    def CA_total_shell(self):
        return self.fc_prod_tank()+self.fc_affa_tank()+self.fc_inj_tank()+self.fc_cmd()+self.FC_environ_shell()